# Import Libraries

In [1]:
'''
Implementation for Experiment the Automatic Noun Compound Interpretion using Deep Neural Networks
'''
import utils
from sklearn import preprocessing
from gensim.models import Word2Vec
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report,confusion_matrix
import pandas as pd
import numpy as np
# Import baseline
from Actual_baseline import Actual_baseline
from Average_baseline import AVG_baseline
from RNN_GRU    import RNN_GRU_baseline
from RNN_GRU_Attention import RNN_GRU_Attention_baseline
import sklearn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import sys
from pycm import *

Using TensorFlow backend.


# Cont declaration

In [2]:
# ***************
# Constant Declaration
# ***************



# Files Paths
type_of_Word2Vec_model = 'CBOW'
vector_file_name = type_of_Word2Vec_model + '_size300_window10_min8'
vector_file_name_path = './../model/' + type_of_Word2Vec_model + '/' + vector_file_name
baseline_train_file_name = 'train_data'
baseline_train_file_path = './../dataset/train_data/' + baseline_train_file_name
Tratz_data_path = '../dataset/Tratz_data/tratz2011_fine_grained_lexical_full/'
class_file = Tratz_data_path + 'classes.txt'
train_data_file = Tratz_data_path + 'train.tsv'
test_data_file = Tratz_data_path + 'test.tsv'
Word2Vec_SG_file_name_path = vector_file_name_path
Word2Vec_CBOW_file_name_path = vector_file_name_path
# Integer Constant
num_of_epoch = 1000
num_of_epoch_composition = 100
batch_size = 1024
batch_size_composition = 128
embedding_dim = 300
num_classes = 37
# Hyperparameters Setup

# Hyper parameters Setup

# Function Implementations

In [3]:
def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

def getClassifierModel(num_of_classes=37,embedding_dim=300,activation_func='tanh',drop_out_rate=0.1):
    model = Sequential()
    # model.add(Dropout(drop_out_rate))
    model.add(Dense(num_of_classes,input_dim = embedding_dim, activation=activation_func))
    model.add(Activation('softmax'))

    print(model.summary())
    #Compile model
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
    # return model
    return model

def readData(data_file,target_dict):
    '''
    Function:
        read the data_file and get the data
    Input:
        data_file(str): data input file name
        target_dict: dictionary of the label dict[label] = class_id(int)
    Output:
        X_word: compound_word
        y: list of class_id
    '''
    # Init return value
    X_word = []
    y = []

    # read the train_data_file
    df = pd.read_csv(data_file,sep='\t',encoding='utf-8')
    df.columns = ['word_1','word_2','label']

    # extract information
    for index, row in df.iterrows():
        line = row['word_1']+'_'+row['word_2']
        X_word.append(line)
        label = target_dict[row['label']]
        y.append(label)

    y_one_hot = [y]

    y_one_hot = indices_to_one_hot(y, num_classes)

    return X_word, y,y_one_hot

def readClassLabel(class_file):
    '''
    Function:
        read the label in class_file and return a dictionary
    Input:
        class_file(str): input file consists of the label list
    Output:
        res(dict):
        Key: label
        value: class_id(int)
    '''
    res = {}

    with open(class_file,'r',encoding='utf-8') as fin:
        for i , line in enumerate(fin):
            res[line.strip()] = i

    return res

def loadWordVecModel(type_of_Word2Vec_model,embedding_dim):
    res = None
    vocab_size = 0
    # is Word2Vec model
    if type_of_Word2Vec_model == 'SG':
        Word2Vec_file_name_path =  Word2Vec_SG_file_name_path
    elif type_of_Word2Vec_model == 'CBOW':
        Word2Vec_file_name_path = Word2Vec_CBOW_file_name_path

    res = Word2Vec.load(Word2Vec_file_name_path) # Load the model from the vector_file_name
    res.wv.init_sims(replace=True)
    vocab_size = len(res.wv.vocab)
    print('Vocab_size: ',vocab_size)
    # is GloVe Model


    # is FastText model

    return res, vocab_size

def wordTovec(X_word,type_of_Word2Vec_model,baseline,word_vec_dict,embedding_dim):
    '''
    Function:
        convert the word into vector by load the Word2Vec model
    Input:
        X_word(list): list of compound word
        each compound_word(list): list of element words
        type_of_Word2Vec_model(str): type of WordVector model (CBOW,SG,GLOVE,FASTTEXT)
    Output:
        X(list): list of compound_word
        each compound_word(list): list of element word_vector
    '''
    # Init return value
    X = []

    X = baseline.predict(X_word,word_vec_dict)
    X = np.array(X)
    return X

In [4]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

# Main

In [5]:
# Main fucntion
    # # Load the pretrained Word Vector
    word_vector, vocab_size = loadWordVecModel(type_of_Word2Vec_model,embedding_dim)


Vocab_size:  2346992


In [6]:
 # Load Tratz data
    target_dict = readClassLabel(class_file)
    X_train_word, _ , y_train = readData(train_data_file,target_dict)
    X_test_word, y_test, _  = readData(test_data_file,target_dict)


In [7]:
# Init Baseline
    baseline    =   Actual_baseline(type_of_Word2Vec_model)
    # baseline    = AVG_baseline(type_of_Word2Vec_model)

    # GRU baseline
    # Load Embedding Matrix for RNN_GRU

    # embedding_matrix = utils.Word2VecTOEmbeddingMatrix(word_vector,embedding_dim)

    # baseline = RNN_GRU_baseline(type_of_Word2Vec_model,vocab_size,embedding_dim,embedding_matrix)

    # X_train_baseline, y_train_baseline = utils.load_data_from_text_file_exclude(baseline_input_train_data,X_test_word,word_vector_dict)
    # Train Baseline
    # baseline.train(X_train_baseline,y_train_baseline,num_of_epoch_composition,batch_size_composition)


    # Use the baseline to convert the word into vector representation
    X_train = wordTovec(X_train_word,type_of_Word2Vec_model,baseline,word_vector,embedding_dim)
    X_test = wordTovec(X_test_word,type_of_Word2Vec_model,baseline,word_vector,embedding_dim)

Compound:  COMPOUND_ID/police_commissioner
In Vocab:  1
Compound:  COMPOUND_ID/army_general
In Vocab:  2
Compound:  COMPOUND_ID/police_general
In Vocab:  3
Compound:  COMPOUND_ID/church_accountant
Compound word not in dictionary
Not in vocab:  1
Compound:  COMPOUND_ID/state_administrator
In Vocab:  4
Compound:  COMPOUND_ID/child_advocate
In Vocab:  5
Compound:  COMPOUND_ID/consumer_advocate
In Vocab:  6
Compound:  COMPOUND_ID/government_agency
In Vocab:  7
Compound:  COMPOUND_ID/state_agency
In Vocab:  8
Compound:  COMPOUND_ID/government_agent
In Vocab:  9
Compound:  COMPOUND_ID/police_agent
In Vocab:  10
Compound:  COMPOUND_ID/committee_aide
Compound word not in dictionary
Not in vocab:  2
Compound:  COMPOUND_ID/government_army
In Vocab:  11
Compound:  COMPOUND_ID/state_assemblyman
In Vocab:  12
Compound:  COMPOUND_ID/district_auditor
In Vocab:  13
Compound:  COMPOUND_ID/government_auditor
In Vocab:  14
Compound:  COMPOUND_ID/state_auditor
In Vocab:  15
Compound:  COMPOUND_ID/district

Not in vocab:  121
Compound:  COMPOUND_ID/investor_optimism
Compound word not in dictionary
Not in vocab:  122
Compound:  COMPOUND_ID/investor_confidence
In Vocab:  489
Compound:  COMPOUND_ID/customer_confidence
In Vocab:  490
Compound:  COMPOUND_ID/consumer_confidence
In Vocab:  491
Compound:  COMPOUND_ID/government_confidence
Compound word not in dictionary
Not in vocab:  123
Compound:  COMPOUND_ID/monetarist_sympathy
Compound word not in dictionary
Not in vocab:  124
Compound:  COMPOUND_ID/voter_apathy
In Vocab:  492
Compound:  COMPOUND_ID/worker_concern
Compound word not in dictionary
Not in vocab:  125
Compound:  COMPOUND_ID/fan_satisfaction
Compound word not in dictionary
Not in vocab:  126
Compound:  COMPOUND_ID/consumer_comfort
Compound word not in dictionary
Not in vocab:  127
Compound:  COMPOUND_ID/patrician_comfort
Compound word not in dictionary
Not in vocab:  128
Compound:  COMPOUND_ID/worker_comfort
Compound word not in dictionary
Not in vocab:  129
Compound:  COMPOUND_ID

Compound:  COMPOUND_ID/consumer_index
Compound word not in dictionary
Not in vocab:  329
Compound:  COMPOUND_ID/employment_index
Compound word not in dictionary
Not in vocab:  330
Compound:  COMPOUND_ID/value_index
In Vocab:  939
Compound:  COMPOUND_ID/confidence_index
In Vocab:  940
Compound:  COMPOUND_ID/banking_index
Compound word not in dictionary
Not in vocab:  331
Compound:  COMPOUND_ID/cash_index
Compound word not in dictionary
Not in vocab:  332
Compound:  COMPOUND_ID/technology_index
Compound word not in dictionary
Not in vocab:  333
Compound:  COMPOUND_ID/market_index
In Vocab:  941
Compound:  COMPOUND_ID/inflation_index
In Vocab:  942
Compound:  COMPOUND_ID/tampon_ad
Compound word not in dictionary
Not in vocab:  334
Compound:  COMPOUND_ID/cow_ad
Compound word not in dictionary
Not in vocab:  335
Compound:  COMPOUND_ID/status_symbol
In Vocab:  943
Compound:  COMPOUND_ID/state_government
In Vocab:  944
Compound:  COMPOUND_ID/township_government
In Vocab:  945
Compound:  COMPO

In Vocab:  1166
Compound:  COMPOUND_ID/market_sector
In Vocab:  1167
Compound:  COMPOUND_ID/science_department
In Vocab:  1168
Compound:  COMPOUND_ID/army_gunfire
Compound word not in dictionary
Not in vocab:  403
Compound:  COMPOUND_ID/consumer_choice
In Vocab:  1169
Compound:  COMPOUND_ID/artillery_support
In Vocab:  1170
Compound:  COMPOUND_ID/market_response
In Vocab:  1171
Compound:  COMPOUND_ID/court_opinion
In Vocab:  1172
Compound:  COMPOUND_ID/state_permit
In Vocab:  1173
Compound:  COMPOUND_ID/state_direction
In Vocab:  1174
Compound:  COMPOUND_ID/coalition_government
In Vocab:  1175
Compound:  COMPOUND_ID/labor_government
In Vocab:  1176
Compound:  COMPOUND_ID/family_reunification
In Vocab:  1177
Compound:  COMPOUND_ID/family_support
In Vocab:  1178
Compound:  COMPOUND_ID/police_gunfire
Compound word not in dictionary
Not in vocab:  404
Compound:  COMPOUND_ID/government_response
In Vocab:  1179
Compound:  COMPOUND_ID/police_response
In Vocab:  1180
Compound:  COMPOUND_ID/bis

Compound:  COMPOUND_ID/genome_project
In Vocab:  1569
Compound:  COMPOUND_ID/parking_meter
In Vocab:  1570
Compound:  COMPOUND_ID/water_meter
In Vocab:  1571
Compound:  COMPOUND_ID/phone_equipment
Compound word not in dictionary
Not in vocab:  566
Compound:  COMPOUND_ID/network_equipment
Compound word not in dictionary
Not in vocab:  567
Compound:  COMPOUND_ID/water_equipment
In Vocab:  1572
Compound:  COMPOUND_ID/crop_chemical
Compound word not in dictionary
Not in vocab:  568
Compound:  COMPOUND_ID/bread_knife
In Vocab:  1573
Compound:  COMPOUND_ID/butter_knife
In Vocab:  1574
Compound:  COMPOUND_ID/water_hose
In Vocab:  1575
Compound:  COMPOUND_ID/water_main
In Vocab:  1576
Compound:  COMPOUND_ID/cargo_aircraft
In Vocab:  1577
Compound:  COMPOUND_ID/tourist_aircraft
Compound word not in dictionary
Not in vocab:  569
Compound:  COMPOUND_ID/cargo_truck
In Vocab:  1578
Compound:  COMPOUND_ID/cargo_space
In Vocab:  1579
Compound:  COMPOUND_ID/cargo_jet
In Vocab:  1580
Compound:  COMPOUN

In Vocab:  2022
Compound:  COMPOUND_ID/market_report
In Vocab:  2023
Compound:  COMPOUND_ID/impact_report
In Vocab:  2024
Compound:  COMPOUND_ID/work_report
In Vocab:  2025
Compound:  COMPOUND_ID/crop_report
In Vocab:  2026
Compound:  COMPOUND_ID/security_report
In Vocab:  2027
Compound:  COMPOUND_ID/crime_report
In Vocab:  2028
Compound:  COMPOUND_ID/lobster_report
Compound word not in dictionary
Not in vocab:  719
Compound:  COMPOUND_ID/malnutrition_report
Compound word not in dictionary
Not in vocab:  720
Compound:  COMPOUND_ID/fish_report
Compound word not in dictionary
Not in vocab:  721
Compound:  COMPOUND_ID/terror_report
Compound word not in dictionary
Not in vocab:  722
Compound:  COMPOUND_ID/vacation_chatter
Compound word not in dictionary
Not in vocab:  723
Compound:  COMPOUND_ID/vacation_magazine
Compound word not in dictionary
Not in vocab:  724
Compound:  COMPOUND_ID/redneck_joke
Compound word not in dictionary
Not in vocab:  725
Compound:  COMPOUND_ID/wine_writer
In Voca

Compound:  COMPOUND_ID/speech_right
In Vocab:  2435
Compound:  COMPOUND_ID/transport_capacity
In Vocab:  2436
Compound:  COMPOUND_ID/appeal_process
In Vocab:  2437
Compound:  COMPOUND_ID/bid_process
In Vocab:  2438
Compound:  COMPOUND_ID/interview_process
In Vocab:  2439
Compound:  COMPOUND_ID/thought_process
In Vocab:  2440
Compound:  COMPOUND_ID/work_process
In Vocab:  2441
Compound:  COMPOUND_ID/research_project
In Vocab:  2442
Compound:  COMPOUND_ID/work_project
In Vocab:  2443
Compound:  COMPOUND_ID/backup_system
In Vocab:  2444
Compound:  COMPOUND_ID/research_system
In Vocab:  2445
Compound:  COMPOUND_ID/transfer_system
In Vocab:  2446
Compound:  COMPOUND_ID/relief_work
In Vocab:  2447
Compound:  COMPOUND_ID/rescue_work
In Vocab:  2448
Compound:  COMPOUND_ID/research_work
In Vocab:  2449
Compound:  COMPOUND_ID/support_work
In Vocab:  2450
Compound:  COMPOUND_ID/rescue_worker
In Vocab:  2451
Compound:  COMPOUND_ID/championship_medal
In Vocab:  2452
Compound:  COMPOUND_ID/cable_net

In Vocab:  2925
Compound:  COMPOUND_ID/court_docket
In Vocab:  2926
Compound:  COMPOUND_ID/customer_account
In Vocab:  2927
Compound:  COMPOUND_ID/employee_right
In Vocab:  2928
Compound:  COMPOUND_ID/family_album
In Vocab:  2929
Compound:  COMPOUND_ID/family_home
In Vocab:  2930
Compound:  COMPOUND_ID/family_estate
In Vocab:  2931
Compound:  COMPOUND_ID/family_heirloom
In Vocab:  2932
Compound:  COMPOUND_ID/family_pet
In Vocab:  2933
Compound:  COMPOUND_ID/family_budget
In Vocab:  2934
Compound:  COMPOUND_ID/family_newspaper
In Vocab:  2935
Compound:  COMPOUND_ID/family_plot
In Vocab:  2936
Compound:  COMPOUND_ID/family_mortuary
Compound word not in dictionary
Not in vocab:  1095
Compound:  COMPOUND_ID/family_fortune
In Vocab:  2937
Compound:  COMPOUND_ID/infantry_vehicle
Compound word not in dictionary
Not in vocab:  1096
Compound:  COMPOUND_ID/nominee_account
In Vocab:  2938
Compound:  COMPOUND_ID/opposition_newspaper
In Vocab:  2939
Compound:  COMPOUND_ID/opposition_stronghold
In V

Not in vocab:  1296
Compound:  COMPOUND_ID/property_issue
In Vocab:  3301
Compound:  COMPOUND_ID/liability_issue
In Vocab:  3302
Compound:  COMPOUND_ID/election_issue
In Vocab:  3303
Compound:  COMPOUND_ID/telecommunication_issue
Compound word not in dictionary
Not in vocab:  1297
Compound:  COMPOUND_ID/settlement_issue
In Vocab:  3304
Compound:  COMPOUND_ID/money_issue
In Vocab:  3305
Compound:  COMPOUND_ID/consumer_issue
In Vocab:  3306
Compound:  COMPOUND_ID/government_issue
In Vocab:  3307
Compound:  COMPOUND_ID/health care_issue
Compound word not in dictionary
Not in vocab:  1298
Compound:  COMPOUND_ID/water_issue
In Vocab:  3308
Compound:  COMPOUND_ID/flag_issue
In Vocab:  3309
Compound:  COMPOUND_ID/family_issue
In Vocab:  3310
Compound:  COMPOUND_ID/character_issue
In Vocab:  3311
Compound:  COMPOUND_ID/utility_issue
Compound word not in dictionary
Not in vocab:  1299
Compound:  COMPOUND_ID/statehood_issue
Compound word not in dictionary
Not in vocab:  1300
Compound:  COMPOUND_

In Vocab:  3695
Compound:  COMPOUND_ID/press_service
In Vocab:  3696
Compound:  COMPOUND_ID/volunteer_service
In Vocab:  3697
Compound:  COMPOUND_ID/guerrilla_action
In Vocab:  3698
Compound:  COMPOUND_ID/media_scrutiny
In Vocab:  3699
Compound:  COMPOUND_ID/press_scrutiny
In Vocab:  3700
Compound:  COMPOUND_ID/creditor_claim
Compound word not in dictionary
Not in vocab:  1399
Compound:  COMPOUND_ID/bulldog_bite
Compound word not in dictionary
Not in vocab:  1400
Compound:  COMPOUND_ID/hyena_bite
Compound word not in dictionary
Not in vocab:  1401
Compound:  COMPOUND_ID/guerrilla_foray
Compound word not in dictionary
Not in vocab:  1402
Compound:  COMPOUND_ID/media_access
In Vocab:  3701
Compound:  COMPOUND_ID/user_access
In Vocab:  3702
Compound:  COMPOUND_ID/doctor_visit
In Vocab:  3703
Compound:  COMPOUND_ID/automobile_pollution
Compound word not in dictionary
Not in vocab:  1403
Compound:  COMPOUND_ID/group_tour
In Vocab:  3704
Compound:  COMPOUND_ID/press_tour
In Vocab:  3705
Comp

In [8]:
# Get Model
    model = getClassifierModel()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 37)                11137     
_________________________________________________________________
activation (Activation)      (None, 37)                0         
Total params: 11,137
Trainable params: 11,137
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
    #Train Model
    model.fit(X_train,y_train,epochs=num_of_epoch , batch_size=batch_size)

Epoch 1/1000
4729/4729 [==============================] - 2s 339us/step - loss: 3.6157 - acc: 0.0190
Epoch 2/1000
4729/4729 [==============================] - 0s 4us/step - loss: 3.5939 - acc: 0.0408
Epoch 3/1000
4729/4729 [==============================] - 0s 4us/step - loss: 3.5717 - acc: 0.0588
Epoch 4/1000
4729/4729 [==============================] - 0s 3us/step - loss: 3.5499 - acc: 0.0780
Epoch 5/1000
4729/4729 [==============================] - 0s 4us/step - loss: 3.5286 - acc: 0.1002
Epoch 6/1000
4729/4729 [==============================] - 0s 4us/step - loss: 3.5084 - acc: 0.1216
Epoch 7/1000
4729/4729 [==============================] - 0s 4us/step - loss: 3.4887 - acc: 0.1391
Epoch 8/1000
4729/4729 [==============================] - 0s 4us/step - loss: 3.4699 - acc: 0.1590
Epoch 9/1000
4729/4729 [==============================] - 0s 5us/step - loss: 3.4519 - acc: 0.1730
Epoch 10/1000
4729/4729 [==============================] - 0s 5us/step - loss: 3.4346 - acc: 0.1903
Epoch 1

Epoch 83/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.9569 - acc: 0.3155
Epoch 84/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.9542 - acc: 0.3161
Epoch 85/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.9515 - acc: 0.3168
Epoch 86/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.9489 - acc: 0.3176
Epoch 87/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.9463 - acc: 0.3170
Epoch 88/1000
4729/4729 [==============================] - 0s 4us/step - loss: 2.9437 - acc: 0.3174
Epoch 89/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.9412 - acc: 0.3174
Epoch 90/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.9387 - acc: 0.3174
Epoch 91/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.9362 - acc: 0.3176
Epoch 92/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.9338 - acc: 0.3182


4729/4729 [==============================] - 0s 5us/step - loss: 2.7295 - acc: 0.3487
Epoch 246/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.7287 - acc: 0.3485
Epoch 247/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.7279 - acc: 0.3489
Epoch 248/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.7271 - acc: 0.3491
Epoch 249/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.7263 - acc: 0.3491
Epoch 250/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.7255 - acc: 0.3489
Epoch 251/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.7248 - acc: 0.3493
Epoch 252/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.7240 - acc: 0.3498
Epoch 253/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.7232 - acc: 0.3500
Epoch 254/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.7224 - acc: 0.3504
Epoch

4729/4729 [==============================] - 0s 5us/step - loss: 2.6323 - acc: 0.3859
Epoch 408/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.6319 - acc: 0.3859
Epoch 409/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.6314 - acc: 0.3861
Epoch 410/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.6310 - acc: 0.3861
Epoch 411/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.6305 - acc: 0.3861
Epoch 412/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.6301 - acc: 0.3861
Epoch 413/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.6297 - acc: 0.3863
Epoch 414/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.6292 - acc: 0.3863
Epoch 415/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.6288 - acc: 0.3868
Epoch 416/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.6284 - acc: 0.3872
Epoch

4729/4729 [==============================] - 0s 5us/step - loss: 2.5735 - acc: 0.4111
Epoch 570/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5732 - acc: 0.4113
Epoch 571/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5729 - acc: 0.4109
Epoch 572/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5726 - acc: 0.4111
Epoch 573/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5723 - acc: 0.4111
Epoch 574/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5720 - acc: 0.4113
Epoch 575/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5717 - acc: 0.4111
Epoch 576/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.5714 - acc: 0.4113
Epoch 577/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.5711 - acc: 0.4113
Epoch 578/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5708 - acc: 0.4113
Epoch

4729/4729 [==============================] - 0s 6us/step - loss: 2.5327 - acc: 0.4295
Epoch 732/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.5325 - acc: 0.4295
Epoch 733/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.5323 - acc: 0.4293
Epoch 734/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.5320 - acc: 0.4293
Epoch 735/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5318 - acc: 0.4295
Epoch 736/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5316 - acc: 0.4299
Epoch 737/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5314 - acc: 0.4297
Epoch 738/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5312 - acc: 0.4299
Epoch 739/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5310 - acc: 0.4299
Epoch 740/1000
4729/4729 [==============================] - 0s 4us/step - loss: 2.5308 - acc: 0.4299
Epoch

4729/4729 [==============================] - 0s 5us/step - loss: 2.5022 - acc: 0.4417
Epoch 894/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5020 - acc: 0.4417
Epoch 895/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5019 - acc: 0.4420
Epoch 896/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.5017 - acc: 0.4420
Epoch 897/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.5015 - acc: 0.4420
Epoch 898/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5014 - acc: 0.4417
Epoch 899/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.5012 - acc: 0.4417
Epoch 900/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5011 - acc: 0.4422
Epoch 901/1000
4729/4729 [==============================] - 0s 5us/step - loss: 2.5009 - acc: 0.4417
Epoch 902/1000
4729/4729 [==============================] - 0s 6us/step - loss: 2.5007 - acc: 0.4415
Epoch

In [10]:
 # Predict
    round_predictions = model.predict_classes(X_test)

In [11]:
# sklearn.utils.multiclass.unique_labels(y_test,round_predictions)

In [12]:
np.set_printoptions(threshold=sys.maxsize)

In [13]:
classes = []
target_names = list(target_dict.keys())
for target_name in target_names:
    classes.append(target_name[:5])



In [14]:
classes = np.array(classes)

In [15]:
# Evaluate
# plot_confusion_matrix(y_test,round_predictions,classes)
report = classification_report(y_test,round_predictions)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.33      0.33      0.33         3
           2       0.00      0.00      0.00         8
           3       0.22      0.04      0.06        56
           4       0.00      0.00      0.00        13
           5       0.25      0.33      0.29         9
           6       0.17      0.06      0.09        62
           9       0.00      0.00      0.00         8
          10       0.11      0.15      0.12        60
          11       0.00      0.00      0.00        13
          12       0.47      0.50      0.48        14
          13       0.00      0.00      0.00         6
          14       0.27      0.63      0.38       156
          15       0.00      0.00      0.00         4
          16       0.33      0.10      0.15        10
          17       0.14      0.06      0.09        47
          18       0.00      0.00      0.00        10
          19       0.00    

/home/an/Env/python3-gpu-tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/an/Env/python3-gpu-tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [16]:
cm = ConfusionMatrix(actual_vector=y_test,predict_vector=round_predictions)

In [20]:
cm.save_html("test",color=(255, 204, 255))

{'Status': True,
 'Message': '/home/an/Compositional_Word_Embedding/old/test.html'}